In [ ]:
!pip install pyxdf

In [1]:
import os
import pyxdf
import pandas as pd
import ast
import numpy as np

In [2]:

folder_path = "./data/Data/"  # Replace with the path to your dataset folder

dataset = list()

# Iterate over files in the folder
for filename in os.listdir(folder_path):
    if filename.endswith(".xdf"):  # Assuming all XDF files have the .xdf extension
        file_path = os.path.join(folder_path, filename)

        # Load XDF data for each file
        streams, file_header = pyxdf.load_xdf(file_path)
        dataset.append(streams)


In [79]:
time_series_list = []
target = []
target_list = [[1,2],[1,8],[2,1],[2,5],[2,9],[3,4],[3,6],[4,3],[4,7],[5,2],[5,5],[5,8],[6,3],[6,7],[7,4],[7,6],[8,1],[8,5],[8,9],[9,2],[9,8]]
marker_number = []
subject_number = []
subject_count = 0
for data in dataset:
    webstream = 0
    marker = 0
    for stream in range(len(data)):
        if data[stream]['info']['name'][0] == "WebcamStream":
           webstream = stream
        if data[stream]['info']['name'][0] == "DecryptMarkerStream":
           marker = stream

    marker_count = 0
    video_count = 0

    for i in range(21):
        start = data[marker]['time_stamps'][marker_count]
        end = data[marker]['time_stamps'][marker_count+1]

        while data[webstream]['time_stamps'][video_count] < start:
            video_count += 1
            continue

        while data[webstream]['time_stamps'][video_count] <= end:
            temp = list()
            for j in data[webstream]['time_series'][video_count]:
                temp.append(j)
            time_series_list.append(temp)
            target.append(target_list[i])
            marker_number.append(i)
            subject_number.append(subject_count)
            video_count += 1

        marker_count += 2
    subject_count += 1

In [81]:
df1 = pd.Series(subject_number)
df2 = pd.Series(marker_number)
df3 = pd.Series(time_series_list)
df4 = pd.Series(target)
df_list = [df1, df2, df3, df4]
df= pd.concat(df_list, axis=1)
df.head(5)

,0,1,2,3
0,0,0,"[296.75113, 297.27917, 297.03198, 294.42212, 2...","[1, 2]"
1,0,0,"[296.2985, 295.68347, 295.629, 293.5011, 296.0...","[1, 2]"
2,0,0,"[295.41623, 296.32166, 296.15216, 294.07843, 2...","[1, 2]"
3,0,0,"[296.1823, 296.2854, 296.07892, 293.95288, 296...","[1, 2]"
4,0,0,"[296.6373, 296.11606, 295.98215, 293.76398, 29...","[1, 2]"


In [82]:
csv_filename = 'trial_data.csv'

# Writing the DataFrame to a CSV file
df.to_csv(csv_filename, index=False)

print(f'Data has been saved to {csv_filename}')

Data has been saved to trial_data.csv


In [11]:
#Analyzing a single xdf file
xdf_file_path = "/DATAHDD/chailex/thesis/eyeTracking/new_data_training/data/Data/sub-P001_ses-S001_task-Default_run-001_eeg_103120.xdf"

streams, file_header = pyxdf.load_xdf(xdf_file_path)

for stream in streams:
    print("Stream name:", stream['info']['name'][0])
    print("Stream type:", stream['info']['type'][0])

    time_series, time_stamps = stream['time_series'], stream['time_stamps']

# Optionally, you can also inspect the file header
print("File header:", file_header)

print(streams[0]['time_stamps'].shape)
print(streams[1]['time_stamps'].shape)
print(streams[2]['time_stamps'].shape)
print(streams[0])
print(len(streams[1]['time_series']))

Stream name: DecryptMarkerStream
Stream type: Markers
Stream name: Mindwave
Stream type: Gaze
Stream name: WebcamStream
Stream type: video
File header: {'info': defaultdict(<class 'list'>, {'version': ['1.0'], 'datetime': ['2023-08-23T10:45:18+0530']})}
(50,)
(56610,)
(1866,)
{'info': defaultdict(<class 'list'>, {'name': ['DecryptMarkerStream'], 'type': ['Markers'], 'channel_count': ['1'], 'channel_format': ['string'], 'source_id': ['myuidw43536'], 'nominal_srate': ['0.000000000000000'], 'version': ['1.100000000000000'], 'created_at': ['311480.7688952000'], 'uid': ['d4a92775-2ede-4316-9a2f-d88262dc0e56'], 'session_id': ['default'], 'hostname': ['DESKTOP-VOI6AV7'], 'v4address': [None], 'v4data_port': ['16573'], 'v4service_port': ['16573'], 'v6address': [None], 'v6data_port': ['16573'], 'v6service_port': ['16573'], 'desc': [None], 'stream_id': 3, 'effective_srate': 0}), 'footer': {'info': defaultdict(<class 'list'>, {'first_timestamp': ['311541.9588719'], 'last_timestamp': ['311604.47016

In [37]:
#This code is to save participant wise data separately. In total we have 57 participants therefore we will get 57 csv files from this block of code. 
time_series_list_notarget = []

marker_number = []
subject_number = []
subject_count = 0

for data in dataset:
    webstream = 0
    marker = 0
    for stream in range(len(data)):
        if data[stream]['info']['name'][0] == "WebcamStream":
           webstream = stream
        if data[stream]['info']['name'][0] == "DecryptMarkerStream":
           marker = stream

    marker_count = 44
    video_count = 0
    escape_count = 0
    subject_time_series_notarget = []
    image_number = 0
    for i in range(3):
        if escape_count == 2:
            escape_count += 2
            marker_count += 2
            continue
        escape_count += 2
        temp_list = []
        start = data[marker]['time_stamps'][marker_count]
        end = data[marker]['time_stamps'][marker_count+1]

        while data[webstream]['time_stamps'][video_count] < start:
            video_count += 1
            continue
        
        while video_count < len(data[webstream]['time_stamps']) and data[webstream]['time_stamps'][video_count] <= end:
            temp_list.append(data[webstream]['time_series'][video_count])
            video_count += 1
        ############################################################
        predict_list = []

        for time_series in temp_list:
            dd = pd.Series(time_series)
            temp = []
            for i in dd:
                temp.append(i)
            predict_list.append(temp)
        
        csv_filename = './participants_57/unknown_data{}_{}.csv'.format(subject_count, image_number)
        image_number += 1 

        df_unknown = pd.Series(predict_list)
        df_unknown.to_csv(csv_filename, index=False)

        print(f'Data has been saved to {csv_filename}')
        #############################################################


        subject_time_series_notarget.append(temp_list)
        marker_count += 2
    time_series_list_notarget.append(subject_time_series_notarget)
    subject_count += 1

Data has been saved to ./participants_57/unknown_data0_0.csv
Data has been saved to ./participants_57/unknown_data0_1.csv
Data has been saved to ./participants_57/unknown_data1_0.csv
Data has been saved to ./participants_57/unknown_data1_1.csv
Data has been saved to ./participants_57/unknown_data2_0.csv
Data has been saved to ./participants_57/unknown_data2_1.csv
Data has been saved to ./participants_57/unknown_data3_0.csv
Data has been saved to ./participants_57/unknown_data3_1.csv
Data has been saved to ./participants_57/unknown_data4_0.csv
Data has been saved to ./participants_57/unknown_data4_1.csv
Data has been saved to ./participants_57/unknown_data5_0.csv
Data has been saved to ./participants_57/unknown_data5_1.csv
Data has been saved to ./participants_57/unknown_data6_0.csv
Data has been saved to ./participants_57/unknown_data6_1.csv
Data has been saved to ./participants_57/unknown_data7_0.csv
Data has been saved to ./participants_57/unknown_data7_1.csv
Data has been saved to .